___
# Insper - Ciência dos Dados - Projeto 1

# Obtenção das avaliações

Utilize este notebook para **baixar avaliações de jogos**.

O código abaixo irá automativamente baixar um conjunto de avaliações reais de jogos submetidas por usuários da Steam.

Será gerado um arquivo `dados.xlsx` contendo as informações disponíveis para o projeto.

## Preparação do ambiente no jupyter

Vamos importar algumas bibliotecas e definir algumas funções úteis!

In [ ]:
pip install urllib3

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from sklearn.utils import shuffle
import os.path
import urllib.request

In [ ]:
def decode(byt):
    mask = b'dataset'
    lmask = len(mask)
    return bytes(c ^ mask[i % lmask] for i, c in enumerate(byt))

def get_data(usuario="user", avaliacoes=500):
    file_path = '../../data/dados.xlsx'
    if not os.path.isfile(file_path):

        print(f'Ok! Vou baixar avaliações de jogos da Steam!')
        print('\nAguarde... Este processo pode demorar alguns minutos!')

        target = decode(b"\x0c\x15\x00\x11\x00_[K\x13\x15\x16]\x02\x1d\x10\t\x01\x03\x06\x16\x11\x16\x02\x1b\x0f\x07\x00\x1a\x10O\x17\x0e\x1eJ\x19\x02\x12\x00\x00\x11\x0c\x18\x01N'\x15\x16\x04\x19 \x00\x00\x00\\\x08\x15\r\x0f[\x12\x07\x00\x15\t>\x1a\x04\x14K\x17\x17\x17").decode('ascii')
        ldf = pd.read_csv(target,usecols=["app_name","review"])
        
        target = decode(b"\x0c\x15\x00\x11\x00_[K\x13\x15\x16]\x02\x1d\x10\t\x01\x03\x06\x16\x11\x16\x02\x1b\x0f\x07\x00\x1a\x10O\x17\x0e\x1eJ\x19\x02\x12\x00\x00\x11\x0c\x18\x01N'\x15\x16\x04\x19 \x00\x00\x00\\\x08\x15\r\x0f[\x12\x07\x00\x15\t>\x04\x0e\x00K\x17\x17\x17").decode('ascii')
        rdf = pd.read_csv(target,usecols=["app_name","review"])

        seed = abs(hash(usuario)) % (10 ** 8)
        np.random.seed(seed)
        perc = np.random.uniform(0.4,0.6)
        left = int(np.ceil(perc * avaliacoes))
        lser = ldf.sample(left,random_state=seed)
        right= avaliacoes-left
        rser = rdf.sample(right,random_state=seed)
        perc_train=0.6

        l_treino = int(np.floor(left * perc_train))
        r_treino = int(np.floor(right * perc_train))

        ltrain = lser.iloc[:l_treino]
        ltest = lser.iloc[l_treino:]
        rtrain = rser.iloc[:r_treino]
        rtest = rser.iloc[r_treino:]

        train = ltrain.append(rtrain,ignore_index = True)
        test = ltest.append(rtest,ignore_index = True)

        train = shuffle(train).reset_index(drop=True)
        test = shuffle(test).reset_index(drop=True)
        
        writer = pd.ExcelWriter(file_path)
        
        train.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)
        
        test.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

        writer.save()

        print('Processo finalizado!')
        print('\nGeramos um arquivo chamado dados.xlsx na pasta data, confira!')

    else:

        raise Exception('Arquivo dados.xlsx já existe na pasta data! Apague o arquivo caso queira gerar novamente!')


## Insira seu usuário Insper

In [ ]:
nome_usuario=""

In [ ]:
nome_usuario = nome_usuario.strip()
if len(nome_usuario) == 0:
    raise Exception('Você precisa inserir seu nome de usuário na variável usuario!')

## Defina a quantidade de avaliações

Defina a quantidade de avaliações a serem baixadas. Lembre-se que a quantidade mínima de avaliações depende da quantidade de categorias que serão avaliadas.

In [ ]:
qtde_avaliacoes = 0

In [ ]:
if qtde_avaliacoes <= 0:
    raise Exception('Você precisa definir a quantidade de avaliações!')

## Obtenção das avaliações

Execute a próxima célula. Ela irá construir uma base de dados personalizada de avaliações.

Este processo poderá demorar alguns minutos. Quando ele finalizar, será gerado um arquivo `dados.xlsx` com as avaliações!

Pontos de atenção:
- Confira se a quantidade de avaliações foi suficiente (conforme enunciado)

In [ ]:
get_data(nome_usuario,qtde_avaliacoes)